# PCA country/domains instead of country/subcat
- show that domains naturally categorize based on censorship
- similar to Ben's analysis of domains vs censorship ratio

In [1]:
from __future__ import division
%matplotlib nbagg
import pandas as pd
import os, sys
import numpy as np
from collections import defaultdict
from caseDetection import detect_case
from world_map_maker import create_world_map
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#import matplotlib as mpl
#mpl.use("GTK3cairo")
#import matplotlib.pyplot as plt
#plt.plot([1,2,3,4],'*-')
from scipy.spatial.distance import cosine
from itertools import cycle

In [43]:
SAMPLENAME = 'Snapshot3'
RESULTS = "results/" + SAMPLENAME + "/"
if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)

# MODEL SETUP
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tsne = TSNE(n_components=2, random_state=0)
pca = PCA(n_components=2)

In [92]:
dom2cat, cat2dom = {}, {}
with open('data/Servers_IMC.txt', 'r') as filep:
    for line in filep.readlines():
        ip, domain, cats = line.strip().split(',')
        cats = cats.split('|')
        dom2cat[domain] = cats
        for cat in cats:
            if cat not in cat2dom:
                cat2dom[cat] = {}
            cat2dom[cat][domain] = True

In [10]:
df_unbiased = pd.read_pickle(RESULTS + 'domain_censorship_results_repeated-categ.pkl')
df_unbiased.head()

,sIP,country,case,domain,subcat
0,1.209.122.23,AE,2,riseup.net,circum
1,1.209.122.23,AF,2,riseup.net,circum
2,1.209.122.23,AL,2,riseup.net,circum
3,1.209.122.23,AM,2,riseup.net,circum
4,1.209.122.23,AO,2,riseup.net,circum


In [11]:
# check for repeated entries [country, domain, subcat, case]
print len(df_unbiased)
print len(df_unbiased.drop_duplicates())
# No repetitions, except multiple subcats

98282
98282


In [14]:
# check domain/country/case reoccured - these should be domains with multiple subcats
df_temp = df_unbiased.groupby(['country','domain','case']).count()
print len(df_temp[df_temp['sIP']>1])
df_temp[df_temp['sIP']>1].head()
# now drop 'subcat' and do PCA on unique entries only

22974


sIP  subcat
country domain         case             
AE      aa.com         2       2       2
        aarp.org       3       3       3
        abc.net.au     2       2       2
        abcnews.go.com 2       3       3
        agoda.com      2       2       2

In [17]:
df_unique = df_unbiased[['country','domain','case']].drop_duplicates()
print len(df_unbiased)
print len(df_unique)

98282
64175


In [44]:
# check if any country/domain has multiple case entries
df_check = pd.DataFrame ( df_unique.groupby(['country', 'domain'])['case'].unique() )
df_check['count'] = df_check['case'].apply(lambda x: len(x))
df_check[df_check['count']>1]

# use first() for now

,,case,count
country,domain,,
AF,torDir,"[1, 2]",2
CA,torDir,"[2, 3]",2
CG,torDir,"[2, 1]",2
CH,torDir,"[2, 1]",2
CU,torDir,"[2, 3]",2
DO,torDir,"[2, 1]",2
ES,torDir,"[2, 1]",2
FI,torDir,"[2, 3]",2
HN,torDir,"[2, 1]",2


In [122]:
df_temp

,country,domain,case,val
0,AE,riseup.net,2,1
1,AF,riseup.net,2,1
2,AL,riseup.net,2,1
3,AM,riseup.net,2,1
4,AO,riseup.net,2,1
5,AR,riseup.net,2,1
6,AT,riseup.net,2,1
7,AU,riseup.net,2,1
8,BA,riseup.net,2,1
9,BD,riseup.net,2,1


In [140]:
# make censorship tables: censorship (Y/N) and technique (case1/case1+3) -- RUN PCA
# 1. make a table of (1 OR 3) only
# 2. make a table of (1 / (1 OR 3)) only
censorship = df_unique.replace({1:1, 2:0, 3:1}).groupby(['domain','country'])['case'].first().unstack()
print len(censorship)
# try dropping na
print len(censorship.dropna())
# doesn't work - fillna with 0.5???

censorship.head()

503
0


country,AE,AF,AL,AM,AO,AR,AT,AU,AZ,BA,...,UA,UG,US,UY,UZ,VE,VN,ZA,ZM,ZW
domain,,,,,,,,,,,,,,,,,,,,,
24hourfitness.com,0,0,1,NaN,0,0,0,0,0,0,...,0,0,NaN,0,0,0,0,1,0,0
6pm.com,0,0,NaN,0,0,0,0,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0
9gag.com,0,0,NaN,NaN,0,0,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa.com,0,0,NaN,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
aarp.org,1,1,0,1,1,0,1,NaN,1,1,...,0,0,NaN,1,NaN,NaN,NaN,NaN,1,NaN


In [51]:
# temp solution
censorship = censorship.fillna(0.5)

In [53]:
# PCA to cluster domains by amount of censorship
df_multidim = censorship
mat = censorship.as_matrix()
df4 = pd.DataFrame(pca.fit_transform(mat)).set_index(df_multidim.index)

df4.head()

,0,1
domain,,
24hourfitness.com,-1.255462,0.430377
6pm.com,-1.967577,-0.226224
9gag.com,-1.423370,0.590247
aa.com,-1.389889,0.823591
aarp.org,6.070237,1.061976


# SVD/PCA
- number of subcats were 17 => 17 dimentional data instead of 2D visualization
- see http://maheshakya.github.io/gsoc/2014/05/18/preparing-a-bench-marking-data-set-using-singula-value-decomposition-on-movielens-data.html

# Represent domain as country: case2 value
- use fillna(0) for NaN in cases => that entry is a no (correct interpretation)
- TODO: we already saw there is NO entry with all three cases NaN, NaN, NaN but might be worth checking again later

In [88]:
#from scipy.sparse.linalg import svds
from numpy.linalg import svd
import scipy.stats as stats
import math

sites = censorship.index
countries = censorship.columns
mat = censorship.T.as_matrix()
#np.rank(mat)
censorship.T.head()

domain,24hourfitness.com,6pm.com,9gag.com,aa.com,aarp.org,abc.net.au,abcnews.go.com,accuweather.com,acs.org,adam4adam.com,...,yellowpages.com,yellowpages.sulekha.com,yelp.com,youporn.com,youtube.com,zappos.com,zara.com,zh.greatfire.org,zillow.com,zomato.com
country,,,,,,,,,,,,,,,,,,,,,
AE,0.0,0.0,0.0,0.0,1,0,0,0.5,0.0,0,...,0,0,0,0,0,0,0.5,1.0,0.5,1
AF,0.0,0.0,0.0,0.0,1,0,0,0.0,0.0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1
AL,1.0,0.5,0.5,0.5,0,0,0,0.5,0.5,0,...,1,1,0,0,0,0,0.0,0.0,0.5,1
AM,0.5,0.0,0.5,0.0,1,0,0,0.0,0.0,0,...,0,0,0,0,0,0,0.5,0.5,0.0,1
AO,0.0,0.0,0.0,0.0,1,0,0,0.0,0.0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1


### Ben's method applied on countries X domains matrix

In [158]:
def center(a, scale=True, axis=0):
    """Center the matrix so that units don't matter by subtracting the mean and
    dividing by the standard deviation
    
    Again, this won't tell us the most commonly censored sites, this will tell us
    which sites are commonly censored together
    
    """
    b = np.array(a, copy=True)
    mean = a.mean(axis=axis)
    b -= mean
    if scale:
        std = b.std(axis=axis)
        std = np.where( std, std, 1. )
        b /= std
    return b

In [86]:
center_obj = center(mat)
U, s, Vt = svd(center_obj)
V = Vt.T

In [87]:
# print some info about the top few principal components
print "Variance explained, mean, std dev of vector"
for index in range(10):
    var_exp = s[index] * s[index] / float(len(countries))
    mean = stats.tmean(V[:, index])
    var = stats.tvar(V[:, index])
    # compute how many elements are really present in the vector
    stuff = [1 for x in V[:, index] if math.fabs(x) > 0.05]
    
    # print it all out
    print "{:< .5e}, {:< .5e}, {:< .5e}, {}".format(var_exp, mean, var, len(stuff))

Variance explained, mean, std dev of vector
 6.28063e+01, -3.49919e-02,  7.65160e-04, 196
 1.84454e+01,  5.83633e-03,  1.95790e-03, 119
 1.81281e+01, -1.55866e-03,  1.98960e-03, 97
 1.42494e+01, -3.03837e-03,  1.98278e-03, 135
 1.20846e+01, -3.21545e-03,  1.98167e-03, 138
 9.79965e+00, -8.41755e-04,  1.99132e-03, 123
 9.00428e+00, -1.34150e-03,  1.99023e-03, 132
 8.53487e+00,  4.79522e-03,  1.96899e-03, 123
 8.11329e+00, -3.91466e-03,  1.97668e-03, 130
 7.65194e+00,  2.06076e-04,  1.99199e-03, 129


In [94]:
import collections
import pprint
# let's explore the individual components in more depth by printing the top 9 components (don't care about first component)
for index in range(0, 10):
    mean = stats.tmean(V[:, index])
    indices = [x for x in range(len(V[:, index])) if math.fabs(V[x, index]) > 0.05]
    #print indices
    site_list = [sites[x] for x in indices]
    counter = collections.Counter()
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            counter[cat] += 1
    print "Component " + str(index) + ": " +", ".join(site_list) + "\n"
    print sorted(counter.items(), key=lambda x: x[1], reverse=True)
    print
    entries = {}
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            if cat not in entries:
                entries[cat] = []
            entries[cat].append(site)
    #pprint.pprint(entries)

Component 0: 9gag.com, accuweather.com, acs.org, adameve.com, adp.com, alkasir.com, allrecipes.com, americanexpress.com, ancestry.com, answers.yahoo.com, apa.org, arxiv.org, asexstories.com, askmen.com, asstr.org, autodesk.com, autotrader.com, aventertainments.com, bankrate.com, battle.net, bedbathandbeyond.com, behance.net, berkeley.edu, bestbuy.com, bettycrocker.com, bgr.com, bhg.com, bhphotovideo.com, booking.com, brainyquote.com, bravenewsoftware.org, britishairways.com, cam4.com, cambridge.org, cancer.org, cdc.gov, chess.com, cnn.com, collegeboard.org, columbia.edu, complex.com, constantcontact.com, consumerreports.org, cornell.edu, costco.com, dict.cc, dieburger.com, digitaltrends.com, docs.google.com, ea.com, easyjet.com, economictimes.indiatimes.com, eonline.com, epa.gov, eqla3.com, espn.go.com, espnfc.com, express-vpn.com, fabswingers.com, fatakat.com, fatwallet.com, fetlife.com, fidelity.com, fifa.com, filgoal.com, fitnessmagazine.com, fixya.com, flickr.com, fontsquirrel.com,

In [159]:
'''
# PCA as 2D figure

mat = df_multidim2.as_matrix()
df4 = pd.DataFrame(tsne.fit_transform(mat)).set_index(df_multidim.index)
fig2, ax2 = plt.subplots(1,1, figsize=(10,10))
ax2.scatter(df4[0], df4[1])
for label, x, y in zip(df4.index, df4[0], df4[1]):
    ax2.annotate(
        label, 
        xy = (x, y), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        #bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
ax2.grid(1)
ax2.set_title("TSNE for server-to-client, client-to-server, and no-blocking for all subcats")
fig2.savefig(currFolder + "tsne_countries_all_cols")
fig2.show()

mat = df_multidim2.as_matrix()
df4 = pd.DataFrame(pca.fit_transform(mat)).set_index(df_multidim.index)
fig2, ax2 = plt.subplots(1,1, figsize=(10,10))
ax2.scatter(df4[0], df4[1])
for label, x, y in zip(df4.index, df4[0], df4[1]):
    ax2.annotate(
        label, 
        xy = (x, y), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        #bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
ax2.grid(1)
ax2.set_title("PCA for server-to-client, client-to-server, and no-blocking for all subcats")
fig2.savefig(currFolder + "pca_countries_all_cols")
#fig2.show()
#plt.close()
'''

'\n# PCA as 2D figure\n\nmat = df_multidim2.as_matrix()\ndf4 = pd.DataFrame(tsne.fit_transform(mat)).set_index(df_multidim.index)\nfig2, ax2 = plt.subplots(1,1, figsize=(10,10))\nax2.scatter(df4[0], df4[1])\nfor label, x, y in zip(df4.index, df4[0], df4[1]):\n    ax2.annotate(\n        label, \n        xy = (x, y), xytext = (-20, 20),\n        textcoords = \'offset points\', ha = \'right\', va = \'bottom\',\n        #bbox = dict(boxstyle = \'round,pad=0.5\', fc = \'yellow\', alpha = 0.5),\n        arrowprops = dict(arrowstyle = \'->\', connectionstyle = \'arc3,rad=0\'))\nax2.grid(1)\nax2.set_title("TSNE for server-to-client, client-to-server, and no-blocking for all subcats")\nfig2.savefig(currFolder + "tsne_countries_all_cols")\nfig2.show()\n\nmat = df_multidim2.as_matrix()\ndf4 = pd.DataFrame(pca.fit_transform(mat)).set_index(df_multidim.index)\nfig2, ax2 = plt.subplots(1,1, figsize=(10,10))\nax2.scatter(df4[0], df4[1])\nfor label, x, y in zip(df4.index, df4[0], df4[1]):\n    ax2.ann

# Represent domain as country: [case1, case2, case3] vector
- use fillna(0) for NaN in cases => that entry is a no (correct interpretation)
- TODO: we already saw there is NO entry with all three cases NaN, NaN, NaN but might be worth checking again later

In [141]:
df_temp = df_unique.copy()
df_temp['val']=1
censorship2 = df_temp.set_index(['case','domain','country'])['val'].unstack(0).fillna(0)

# check if something with NO entries
tot = censorship2.sum(axis=1)
tot[tot<1]

Series([], dtype: float64)

In [153]:
# lots of NaN => untested OR not seen just fill them with 0
censorship3 = censorship2.unstack().fillna(0)
censorship3.head()

case               1                            ...  3                       \
country           AE AF AL AM AO AR AT AU AZ BA ... UA UG US UY UZ VE VN ZA   
domain                                          ...                           
24hourfitness.com  0  0  1  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0   
6pm.com            0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0   
9gag.com           0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0   
aa.com             0  0  0  0  0  0  0  0  0  0 ...  1  0  0  0  0  0  0  0   
aarp.org           0  1  0  1  1  0  1  0  1  1 ...  0  0  0  1  0  0  0  0   

case                     
country           ZM ZW  
domain                   
24hourfitness.com  0  0  
6pm.com            0  0  
9gag.com           0  0  
aa.com             0  0  
aarp.org           1  0  

[5 rows x 426 columns]

In [154]:
sites = censorship3.index
country_case = censorship3.columns
mat = censorship3.T.as_matrix()
#np.rank(mat)
censorship3.T.head()

domain        24hourfitness.com  6pm.com  9gag.com  aa.com  aarp.org  \
case country                                                           
1    AE                       0        0         0       0         0   
     AF                       0        0         0       0         1   
     AL                       1        0         0       0         0   
     AM                       0        0         0       0         1   
     AO                       0        0         0       0         1   

domain        abc.net.au  abcnews.go.com  accuweather.com  acs.org  \
case country                                                         
1    AE                0               0                0        0   
     AF                0               0                0        0   
     AL                0               0                0        0   
     AM                0               0                0        0   
     AO                0               0                0        0   

domain        adam4adam.com     ...      yellowpages.com  \
case country                    ...                        
1    AE                   0     ...                    0   
     AF                   1     ...                    0   
     AL                   0     ...                    1   
     AM                   0     ...                    0   
     AO                   0     ...                    0   

domain        yellowpages.sulekha.com  yelp.com  youporn.com  youtube.com  \
case country                                                                
1    AE                             0         0            0            0   
     AF                             0         0            0            0   
     AL                             1         0            0            0   
     AM                             0         0            0            0   
     AO                             0         0            0            0   

domain        zappos.com  zara.com  zh.greatfire.org  zillow.com  zomato.com  
case country                                                                  
1    AE                0         0                 1           0           1  
     AF                0         0                 0           0           1  
     AL                0         0                 0           0           1  
     AM                0         0                 0           0           1  
     AO                0         0                 0           1           1  

[5 rows x 503 columns]

In [155]:
center_obj = center(mat)
U, s, Vt = svd(center_obj)
V = Vt.T

# print some info about the top few principal components
print "Variance explained, mean, std dev of vector"
for index in range(10):
    var_exp = s[index] * s[index] / float(len(countries))
    mean = stats.tmean(V[:, index])
    var = stats.tvar(V[:, index])
    # compute how many elements are really present in the vector
    stuff = [1 for x in V[:, index] if math.fabs(x) > 0.05]
    
    # print it all out
    print "{:< .5e}, {:< .5e}, {:< .5e}, {}".format(var_exp, mean, var, len(stuff))

Variance explained, mean, std dev of vector
 1.01900e+03, -3.73341e-02,  5.95421e-04, 72
 1.14915e+02,  1.95347e-02,  1.60967e-03, 73
 1.71198e+01, -1.52436e-03,  1.98970e-03, 89
 1.52813e+01,  4.88363e-03,  1.96813e-03, 138
 1.25439e+01, -6.18505e-03,  1.95370e-03, 20
 1.00013e+01,  1.35941e-03,  1.99018e-03, 122
 6.88657e+00, -2.38619e-03,  1.98633e-03, 169
 6.41538e+00, -3.96171e-04,  1.99187e-03, 92
 5.78334e+00,  1.13911e-03,  1.99073e-03, 95
 5.60376e+00,  2.50978e-04,  1.99197e-03, 104


In [156]:
# let's explore the individual components in more depth by printing the top 9 components (don't care about first component)
for index in range(0, 10):
    mean = stats.tmean(V[:, index])
    indices = [x for x in range(len(V[:, index])) if math.fabs(V[x, index]) > 0.05]
    #print indices
    site_list = [sites[x] for x in indices]
    counter = collections.Counter()
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            counter[cat] += 1
    print "Component " + str(index) + ": " +", ".join(site_list) + "\n"
    print sorted(counter.items(), key=lambda x: x[1], reverse=True)
    print
    entries = {}
    for site in site_list:
        cats = dom2cat[site]
        for cat in cats:
            if cat not in entries:
                entries[cat] = []
            entries[cat].append(site)
    #pprint.pprint(entries)

Component 0: 6pm.com, alarabiya.net, alkasir.com, askmen.com, autodesk.com, bdr130.net, bom.gov.au, bravenewsoftware.org, cafemom.com, cbc.ca, copyscape.com, cricket.co.za, dailykos.com, debonairblog.com, dpreview.com, ea.com, economictimes.indiatimes.com, familysearch.org, fatakat.com, fetlife.com, fontsquirrel.com, gamesradar.com, girlsgogames.com, gq.com, groupon.com, hindustantimes.com, history.com, hotwire.com, howstuffworks.com, hulu.com, indianrail.gov.in, jsoftj.com, kooora.com, kyknet.dstv.com, leagueoflegends.com, m5zn.com, maktoob.com, marriott.com, mbc.net, menshealth.com, mercola.com, merriam-webster.com, metacritic.com, mrskin.com, myfitnesspal.com, noaa.gov, online.citibank.com, opendns.com, payserve.com, pcgamer.com, phonearena.com, pinterest.com, planetsuzy.org, popsugar.com, prevention.com, psychologytoday.com, salon.com, sherdog.com, shutterstock.com, smh.com.au, snopes.com, state.gov, thesaurus.com, timeout.com, timesofindia.indiatimes.com, tmz.com, ucla.edu, verizo

#TODO: Represent domain as country: [case1, case2, case3, DNS] vector
- use fillna(0) for NaN in cases => that entry is a no (correct interpretation)
- TODO: we already saw there is NO entry with all three cases NaN, NaN, NaN but might be worth checking again later